## Prep

In [3]:
import numpy as np
import pandas as pd
import os
import re

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
processed_folder = '/content/drive/MyDrive/UIT/UIT_Graduation_Thesis/Dataset/1_Processed/CSI_Filtered_Amp'
raw_folder_ss1 = '/content/drive/MyDrive/UIT/UIT_Graduation_Thesis/Dataset/0_Raw/2024-09-28/CSI_Packets/session_1'
raw_folder_ss2 = '/content/drive/MyDrive/UIT/UIT_Graduation_Thesis/Dataset/0_Raw/2024-09-28/CSI_Packets/session_2'

# List all files in the folder
processed_list = [f for f in os.listdir(processed_folder) if f.endswith('.csv')]

dfs = []
for file_name in processed_list:
    match = re.search(r"csi-filtered-d1-sess0(\d+)-(\d+)", file_name)
    if match:
      print(file_name)
      session = match.group(1)
      label = match.group(2)
      print("label:", label)
      file_path = os.path.join(processed_folder, file_name)
      df = pd.read_csv(file_path)

      raw_folder = raw_folder_ss1 if session == '1' else raw_folder_ss2
      raw_file_name = f'{label}_persons{"_cont" if session == "2" else ""}.csv'
      raw_file_path = os.path.join(raw_folder, raw_file_name)

      if os.path.exists(raw_file_path):
        rssi_df = pd.read_csv(raw_file_path, usecols=['rssi'])
        if not rssi_df.empty:  # Check if rssi_df is not empty
          df["rssi"] = rssi_df["rssi"]
      df["label"] = int(label)
      dfs.append(df)
print("Number of dataframes loaded: ", len(dfs))

csi-filtered-d1-sess02-0.csv
label: 0
csi-filtered-d1-sess02-1.csv
label: 1
csi-filtered-d1-sess02-2.csv
label: 2
csi-filtered-d1-sess02-3.csv
label: 3
csi-filtered-d1-sess02-4.csv
label: 4
csi-filtered-d1-sess02-5.csv
label: 5
csi-filtered-d1-sess01-0.csv
label: 0
csi-filtered-d1-sess01-1.csv
label: 1
csi-filtered-d1-sess01-2.csv
label: 2
csi-filtered-d1-sess01-3.csv
label: 3
csi-filtered-d1-sess01-4.csv
label: 4
csi-filtered-d1-sess01-5.csv
label: 5
Number of dataframes loaded:  12


In [ ]:
print(dfs[5].shape)

(16413, 54)


In [ ]:
print(dfs[5].head(5))

          0         1          2          3          4          5          6  \
0  6.856949  5.269875   7.714059   8.905305   8.597433  10.339726  12.420630   
1  7.911175  7.323215   8.941853  11.844399  10.477811  11.429932  14.011089   
2  8.707656  8.641500  10.204781  13.145867  11.801256  12.291970  14.955239   
3  9.233672  9.336433  11.365571  13.224723  12.621846  12.922443  15.350582   
4  9.476501  9.519715  12.286951  12.495981  12.993661  13.317955  15.294616   

           7          8          9  ...         44         45         46  \
0  11.518278  11.942980  13.156781  ...  21.828870  21.556131  20.271051   
1  13.766368  14.628056  17.159597  ...  25.920977  26.293710  25.953205   
2  14.982780  16.366386  19.455177  ...  28.387842  29.419357  29.412523   
3  15.383103  17.269861  20.353092  ...  29.484460  31.131224  31.017698   
4  15.182922  17.450370  20.162910  ...  29.465820  31.627470  31.137420   

          47         48         49         50         51  rssi

## Feature Extraction

### Segmentation and extraction

In [10]:
def segment_data(data, window_size=200, overlap=50):
    """
    Segments data into overlapping windows.

    Args:
        data (numpy.ndarray): The data to segment (CSI or RSS).
        window_size (int): Size of each window in samples.
        overlap (int): Overlap between consecutive windows in samples.

    Returns:
        list: A list of segmented data windows.
    """

    segmented_data = []
    for i in range(0, data.shape[0] - window_size + 1, window_size - overlap):
        window = data[i:i+window_size] if data.ndim == 1 else data[i:i+window_size, :]
        segmented_data.append(window)
    return segmented_data

In [ ]:
def extract_features(csi_bundle):
    """
    Args:
        csi_bundle: numpy array containing CSI amplitude data (shape: [packets, subcarriers])

    Returns:
        A dictionary containing the calculated statistics.
    """
    features = {}
    for i in range(csi_bundle.shape[1] - 1): # Loop through each subcarrier
      features[f'std_subcarrier_{i}'] = np.std(csi_bundle[:, i]) # Standard deviation
      features[f'mean_subcarrier_{i}'] = np.mean(csi_bundle[:, i]) # The average amplitude value
      features[f'max_subcarrier_{i}'] = np.max(csi_bundle[:, i])
      features[f'min_subcarrier_{i}'] = np.min(csi_bundle[:, i])
      features[f'qtu_subcarrier_{i}'] = np.percentile(csi_bundle[:, i], 75) # Upper quartile
      features[f'qtl_subcarrier_{i}'] = np.percentile(csi_bundle[:, i], 25) # Lower quartile
      features[f'iqr_subcarrier_{i}'] = features[f'qtu_subcarrier_{i}'] - features[f'qtl_subcarrier_{i}']
      if i >= 2 and i <= 49:
        diff = 0
        for j in range(1,3):  # Ignore first and last two subcarriers
            diff += np.sum(
                np.abs(csi_bundle[:, i] - csi_bundle[:, i - j]) + np.abs(csi_bundle[:, i + j] - csi_bundle[:, i])
          )
        features[f'adj_subcarrier_{i-2}'] = diff

    euclidean_distances = []
    for i in range(1, csi_bundle.shape[0]):  # Loop through packets starting from the second
      distances = np.linalg.norm(csi_bundle[i, :] - csi_bundle[i-1, :], axis=0)
      euclidean_distances.append(distances)

    features['euc'] = np.median(euclidean_distances)
    features['rss_std']= np.std(csi_bundle[:, -1])
    features = pd.DataFrame([features])
    return features

In [12]:
# Segementation and Extraction begin here
features_list = []
features_labels = []
for df in dfs:
  print(f"Extracting features for CSI data of {df['label'][0]} people")
  temp_df = df.copy()
  temp_df = temp_df.drop("label", axis=1)
  temp_np_array = np.array(temp_df) # Turn into numpy array for easier matrix calculation
  segments = segment_data(temp_np_array)
  print(f"There are {len(segments)} segments to extract")
  for segment in segments:
    features = extract_features(segment)
    features_list.append(features)
    features_labels.append(df['label'][0])

Extracting features for CSI data of 0 people
There are 104 segments to extract
Extracting features for CSI data of 1 people
There are 108 segments to extract
Extracting features for CSI data of 2 people
There are 108 segments to extract
Extracting features for CSI data of 3 people
There are 101 segments to extract
Extracting features for CSI data of 4 people
There are 104 segments to extract
Extracting features for CSI data of 5 people
There are 109 segments to extract
Extracting features for CSI data of 0 people
There are 119 segments to extract
Extracting features for CSI data of 1 people
There are 106 segments to extract
Extracting features for CSI data of 2 people
There are 105 segments to extract
Extracting features for CSI data of 3 people
There are 104 segments to extract
Extracting features for CSI data of 4 people
There are 103 segments to extract
Extracting features for CSI data of 5 people
There are 89 segments to extract


In [ ]:
print(len(features_list))

935


In [ ]:
print(features_list[0])

   std_subcarrier_0  mean_subcarrier_0  max_subcarrier_0  min_subcarrier_0  \
0          2.425237           19.55163         26.262447         12.873449   

   qtu_subcarrier_0  qtl_subcarrier_0  iqr_subcarrier_0  std_subcarrier_1  \
0         21.231588         18.050487          3.181101          2.142136   

   mean_subcarrier_1  max_subcarrier_1  ...  iqr_subcarrier_50  \
0          19.075911         24.315708  ...           1.908455   

   std_subcarrier_51  mean_subcarrier_51  max_subcarrier_51  \
0           1.661191           19.363485           27.38048   

   min_subcarrier_51  qtu_subcarrier_51  qtl_subcarrier_51  iqr_subcarrier_51  \
0          15.994982          20.238808          18.216065           2.022742   

        euc   rss_std  
0  4.369358  1.645532  

[1 rows x 414 columns]


### Export features

In [ ]:
label_groups = {}

# Group features by label
for features, label in zip(features_list, features_labels):
    if label not in label_groups:
        label_groups[label] = []
    label_groups[label].append(features)

# Concatenate and export DataFrames for each label
for label, dfs in label_groups.items():
    concatenated_df = pd.concat(dfs, ignore_index=True)
    name = f"csi-features-full-d1-{label}.csv"
    concatenated_df.to_csv(f'/content/drive/MyDrive/UIT/UIT_Graduation_Thesis/Dataset/1_Processed/CSI_Features/{name}', index=False)